# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('weatherdata.csv')
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Ayora,-0.7393,-90.3518,68.00,83,40,9.24,EC,1632191084
1,1,Yulara,-25.2406,130.9889,60.64,29,6,17.27,AU,1632191279
2,2,Chui,-33.6971,-53.4616,54.43,88,84,16.78,UY,1632191104
3,3,Bredasdorp,-34.5322,20.0403,45.72,87,15,2.30,ZA,1632191109
4,4,Turaif,31.6725,38.6637,68.25,42,0,8.05,SA,1632191613


In [3]:
gmaps.configure(api_key=g_key)

locations = weather_data[['Lat','Lng']]
humidities = weather_data['Humidity']


In [4]:
fig = gmaps.figure()

try:
    heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                     dissipating=False, max_intensity=100,
                                     point_radius = 2)
except:
    print('There was an error with the Google heatmap API call.')
    
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Ideal Conditions
low = 72
high = 79
wind_speed_max = 10
cloudiness_max = 0

ideal_data = weather_data[(weather_data['Wind Speed'] <= wind_speed_max) & (weather_data['Cloudiness'] <= cloudiness_max) & (weather_data['Max Temp'] <= high)& (weather_data['Max Temp'] >= low)]
ideal_data = ideal_data.dropna()

ideal_data.head(20)





,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,Nizwá,22.9333,57.5333,78.13,70,0,0.74,OM,1632191212
100,100,Sakakah,29.9697,40.2064,78.37,23,0,9.22,SA,1632191171
220,220,Cayenne,4.9333,-52.3333,77.04,94,0,0.00,GF,1632191202
370,370,Abu Samrah,35.3029,37.1841,72.84,45,0,5.48,SY,1632191742
419,419,Kiryat Gat,31.6100,34.7642,74.08,76,0,4.43,IL,1632190942
424,424,Darnah,32.7670,22.6367,78.96,49,0,4.41,LY,1632191763
504,504,Bani Walid,31.7566,13.9942,77.16,30,0,2.64,LY,1632191794
505,505,Yanta,36.2411,115.6672,77.07,62,0,5.64,CN,1632191795


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_data
hotel_df['Hotel Name'] = 'error: none found'

target_type = 'lodging'
radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    location = f"{row['Lat']},{row['Lng']}"
    params ={
        'location':location,
        'type':target_type,
        'radius':radius,
        'key':g_key
    }
    
    
    response = requests.get(base_url, params).json()
    try:
        hotel_name = response["results"][0]["name"]
        print(f"City: {row['City']}")
        print(f'Nearest Hotel: {hotel_name}')
        print('________________________________________')
        hotel_df.loc[index, ['Hotel Name']] = hotel_name
    except:
        print(f"City: {row['City']}")
        print('error: No Hotel Found')
        print('________________________________________')
          





City: Nizwá
Nearest Hotel: Orient Apartments
________________________________________
City: Sakakah
Nearest Hotel: Fakhamat Al Orjoana Apartment
________________________________________
City: Cayenne
Nearest Hotel: Hôtel Le Dronmi
________________________________________
City: Abu Samrah
error: No Hotel Found
________________________________________
City: Kiryat Gat
Nearest Hotel: OlusHome
________________________________________
City: Darnah
Nearest Hotel: فندق الفردوس
________________________________________
City: Bani Walid
Nearest Hotel: منزل الصابر أبوفانة
________________________________________
City: Yanta
Nearest Hotel: Xinhe Hotel
________________________________________


In [7]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
83,83,Nizwá,22.9333,57.5333,78.13,70,0,0.74,OM,1632191212,Orient Apartments
100,100,Sakakah,29.9697,40.2064,78.37,23,0,9.22,SA,1632191171,Fakhamat Al Orjoana Apartment
220,220,Cayenne,4.9333,-52.3333,77.04,94,0,0.00,GF,1632191202,Hôtel Le Dronmi
370,370,Abu Samrah,35.3029,37.1841,72.84,45,0,5.48,SY,1632191742,error: none found
419,419,Kiryat Gat,31.6100,34.7642,74.08,76,0,4.43,IL,1632190942,OlusHome
424,424,Darnah,32.7670,22.6367,78.96,49,0,4.41,LY,1632191763,فندق الفردوس
504,504,Bani Walid,31.7566,13.9942,77.16,30,0,2.64,LY,1632191794,منزل الصابر أبوفانة
505,505,Yanta,36.2411,115.6672,77.07,62,0,5.64,CN,1632191795,Xinhe Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
# Create symbol layer
layer = gmaps.marker_layer(
    locations, 
    info_box_content=hotel_info
)

# Display figure

fig.add_layer(layer)

fig


Figure(layout=FigureLayout(height='420px'))